## Benchmarking Perceptron



## Amazon book review

In [1]:
x = rand(10000)
y = rand(10000);

In [2]:
using BenchmarkTools

path_to_data="../../TestingData/SentimentAnalysis/"
push!(LOAD_PATH,path_to_data)
using AmazonBookReviews

path_to_perceptron ="../source/"
push!(LOAD_PATH,path_to_perceptron)
using MulticlassPerceptron

In [3]:
peakflops()

7.394539573424448e10

In [4]:
versioninfo()

Julia Version 0.6.0
Commit 903644385b (2017-06-19 13:05 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin13.4.0)
  CPU: Intel(R) Xeon(R) CPU E5-1620 v2 @ 3.70GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Sandybridge)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, ivybridge)


### Testing the percep Sparse data Amazon book review



In [5]:
path_to_data="../../TestingData/SentimentAnalysis/"
push!(LOAD_PATH,path_to_data)
using AmazonBookReviews

In [6]:
aux = AmazonBookReviews.load_data(;path=path_to_data, min_support=1)

min support:1

(Dict("shot_by"=>1,"charlotte_hughes"=>2,"justice_must"=>3,"language_gage"=>4,"replacing_the"=>5,"various_scribes"=>6,"interesting_sections"=>7,"destroy_recommended"=>8,"story's_decent"=>9,"suspense_starts"=>10…), Dict(119533=>"quant-heavy",193273=>"of_destruction",43031=>"audience_in",127295=>"of_junk",61670=>"ingledew_a",119601=>"placement_and",136672=>"imagination_before",104394=>"worse_reinhart",189695=>"of_object",179937=>"sees_europe"…), Dict("shot_by"=>1,"charlotte_hughes"=>1,"justice_must"=>1,"language_gage"=>1,"replacing_the"=>1,"various_scribes"=>1,"interesting_sections"=>1,"destroy_recommended"=>1,"story's_decent"=>1,"suspense_starts"=>1…), (Int16[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]))

In [7]:
X_train, y_train = aux[4]

(Int16[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [8]:
X_train = Array{Float32}(X_train);

In [9]:
#X_train_sp = sparse(X_train);
X_train_sp = SparseMatrixCSC{Float32,Int32}(Array{Int32}(X_train);)
y_train[y_train.==-1] = 2
y_train = Array{Int32}(y_train);

In [10]:
size(X_train[:,1]), size(X_train_sp[:,1]),

((195887,), (195887,))

# Sparse operations vs dense

In [10]:
typeof(X_train_sp), typeof(X_train)

(SparseMatrixCSC{Float32,Int32}, Array{Float32,2})

### Dense data and sparse weights

In [12]:
n_features = size(X_train)[1]
percep = MulticlassPerceptron.MPerceptron(Float32, 2, n_features)
typeof(percep.W), typeof(percep.b)

(Array{Float32,2}, Array{Float32,1})

In [13]:
# Dense weights and dense data
@benchmark percep.W'* X_train[:,1] + percep.b

BenchmarkTools.Trial: 
  memory estimate:  765.47 KiB
  allocs estimate:  5
  --------------
  minimum time:     167.029 μs (0.00% GC)
  median time:      387.072 μs (0.00% GC)
  mean time:        458.227 μs (16.97% GC)
  maximum time:     4.817 ms (81.66% GC)
  --------------
  samples:          10000
  evals/sample:     1

### Sparse data and sparse weights

In [14]:
n_features = size(X_train)[1]
percep = MulticlassPerceptron.MPerceptron(Float32, 2, n_features)
typeof(percep.W), typeof(percep.b)

(Array{Float32,2}, Array{Float32,1})

In [15]:
percep.W = sparse(percep.W)
percep.b = sparse(percep.b);

In [16]:
# Sparse data, dense weights
@benchmark percep.W'* X_train_sp[:,1] + percep.b

BenchmarkTools.Trial: 
  memory estimate:  1.92 KiB
  allocs estimate:  18
  --------------
  minimum time:     252.585 μs (0.00% GC)
  median time:      272.165 μs (0.00% GC)
  mean time:        314.771 μs (0.13% GC)
  maximum time:     4.766 ms (85.36% GC)
  --------------
  samples:          10000
  evals/sample:     1

### Benchmarking Perceptron learning dense weights and data

In [17]:
n_features = size(X_train)[1]
percep = MulticlassPerceptron.MPerceptron(Float32, 2, n_features)
typeof(percep.W), typeof(percep.b), typeof(X_train)

(Array{Float32,2}, Array{Float32,1}, Array{Float32,2})

In [18]:
@benchmark percep.W[:,2]

BenchmarkTools.Trial: 
  memory estimate:  765.28 KiB
  allocs estimate:  3
  --------------
  minimum time:     82.489 μs (0.00% GC)
  median time:      204.954 μs (0.00% GC)
  mean time:        278.918 μs (22.98% GC)
  maximum time:     3.816 ms (75.97% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [19]:
m = 23
y = y_train
x = view(X_train,:,m)
learning_rate  = Float32(0.1)
@benchmark percep.W[:,2] .= percep.W[:, y[m]]  .+ learning_rate .* x

BenchmarkTools.Trial: 
  memory estimate:  765.94 KiB
  allocs estimate:  35
  --------------
  minimum time:     177.491 μs (0.00% GC)
  median time:      348.252 μs (0.00% GC)
  mean time:        398.303 μs (18.27% GC)
  maximum time:     3.575 ms (77.02% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [20]:
typeof(percep.W), typeof(percep.b), typeof(X_train), typeof(X_train)

(Array{Float32,2}, Array{Float32,1}, Array{Float32,2}, Array{Float32,2})

In [21]:
@time fit!(percep, X_train, y_train; 
           n_epochs=2, print_flag=true)

  3.799133 seconds (3.23 M allocations: 2.634 GiB, 9.80% gc time)


### Benchmarking Perceptron learning with sparse weights and data

In [22]:
n_features = size(X_train_sp)[1]
percep = MulticlassPerceptron.MPerceptron(Float32, 2, n_features)
percep.W = sparse(percep.W)
percep.b = sparse(percep.b)
typeof(percep.W), typeof(percep.b), typeof(X_train)

(SparseMatrixCSC{Float32,Int64}, SparseVector{Float32,Int64}, Array{Float32,2})

In [23]:
typeof(percep.W), typeof(X_train_sp), typeof(X_train_sp[:,2])

(SparseMatrixCSC{Float32,Int64}, SparseMatrixCSC{Float32,Int32}, SparseVector{Float32,Int32})

In [24]:
@benchmark percep.W[:,2]

BenchmarkTools.Trial: 
  memory estimate:  2.24 MiB
  allocs estimate:  5
  --------------
  minimum time:     163.724 μs (0.00% GC)
  median time:      550.886 μs (0.00% GC)
  mean time:        958.576 μs (27.91% GC)
  maximum time:     6.288 ms (60.80% GC)
  --------------
  samples:          5153
  evals/sample:     1

In [25]:
m=23
y = y_train
x = view(X_train_sp,:,m)
learning_rate  = Float32(0.1)
@benchmark percep.W[:,2] .=percep.W[:, y[m]]  .+ learning_rate .* x

BenchmarkTools.Trial: 
  memory estimate:  2.24 MiB
  allocs estimate:  37
  --------------
  minimum time:     22.656 ms (0.00% GC)
  median time:      24.235 ms (0.00% GC)
  mean time:        25.315 ms (1.05% GC)
  maximum time:     38.853 ms (0.00% GC)
  --------------
  samples:          198
  evals/sample:     1

In [26]:
@time fit!(percep, X_train_sp, y_train;
           n_epochs=2, print_flag=true)

146.168707 seconds (2.03 M allocations: 860.671 MiB, 0.12% gc time)


#### Fast weights * data predictions


How can we make the prediction ```h.w * x + h.b``` fast?

- Can we have non sparse weights and sparse data fast prediccions?


#### About storing sparse data as sparse matrix

The data can is stored as 
```SparseMatrixCSC{Int16,Int64}```

